In [1]:
import torch
import os
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR='/content/drive/MyDrive/BIR_Workshop/model_mesh'
PATH=os.path.join(DATA_DIR,'best_model_3')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import drive
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import torchtext
from torch.utils.data import Dataset,DataLoader
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

from sklearn.metrics import classification_report as cr

device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')

NUM_CLASSES = 5 # 195
BATCH_SIZE = 32

In [3]:
class MESHDataset(Dataset):
    def __init__(self,numpy_file,label_file):
     
      try:
        self.data= np.load(numpy_file)
        self.labels = np.load(label_file)
      except Exception as err:
        raise Exception(f'ERROR OPENING FILES: {numpy_file} | {label_file}. See Error below. \n {err}')  
       
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        #Get the element with `idx`
        #Output an 89*89 matrix
        return self.data[idx].flatten(), self.labels[idx]

In [4]:
test_dataset = MESHDataset(os.path.join(DATA_DIR,'test.npy'),os.path.join(DATA_DIR,'grouped_test_labels.npy'))
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
from sklearn.metrics import accuracy_score
def compute_accuracy(pred, target):
  return target.detach().numpy(), pred.argmax(-1).detach().numpy()

In [6]:
class BaselineModel(nn.Module):

    def __init__(self, matrix_size=89):
        super(BaselineModel, self).__init__()
        self.linear1 = nn.Linear(matrix_size**2, (matrix_size**2)//2)
        self.linear2 = nn.Linear((matrix_size**2)//2, matrix_size**2//4)
        self.linear3 = nn.Linear((matrix_size**2)//4, NUM_CLASSES)
        
    def forward(self, x):

        x = self.linear1(x)
        x = F.relu(x) 

        x = self.linear2(x)
        x = F.relu(x)

        x = self.linear3(x)
        return x

In [8]:
model = BaselineModel().to(device)
model.load_state_dict(torch.load(PATH, map_location='cpu'))
print(model)
model.eval()

labels, preds = [], []
for test_features, test_label in test_dataloader:
  # print(test_features.size())
  test_features, test_label = test_features.to(device), test_label.to(device)
  test_pred = model(test_features.float())
  true, predictions = compute_accuracy(test_pred.cpu(),test_label.cpu())
  labels += true.tolist()
  preds += predictions.tolist()
print(f'Best Dense Model Accuracy with 5 classes:{accuracy_score(labels, preds)}')
print("Best Dense Model Weighted F1-Score with 5 classes: {}".format(f1_score(labels, preds, average='weighted')))

BaselineModel(
  (linear1): Linear(in_features=7921, out_features=3960, bias=True)
  (linear2): Linear(in_features=3960, out_features=1980, bias=True)
  (linear3): Linear(in_features=1980, out_features=5, bias=True)
)
Best Dense Model Accuracy with 5 classes:0.8308586184635248
Best Dense Model Weighted F1-Score with 5 classes: 0.8293958287174851
